In [1]:
import random
import copy
import pprint
import time

In [23]:
mttca = 345
n = 6
mttbr = 10
alpha = 1/mttca
beta = 1/mttbr

In [24]:
matrix = [[1-n*alpha, n*alpha, 0],
          [beta, 1-beta-alpha*(n-1), alpha*(n-1)],
          [0,0,1]]
pprint.pprint(matrix)

[[0.9826086956521739, 0.017391304347826087, 0],
 [0.1, 0.8855072463768117, 0.014492753623188406],
 [0, 0, 1]]


In [25]:
matrix_intervals = copy.deepcopy(matrix)
for x in matrix_intervals:
    for j in range(len(x)):
        if j!=0:
            x[j] += x[j-1]
    x[-1] = 1.0
pprint.pprint(matrix_intervals)

[[0.9826086956521739, 1.0, 1.0], [0.1, 0.9855072463768116, 1.0], [0, 0, 1.0]]


In [6]:
class MyFiniteStateMachine(object):
    def __init__(self, matrix):
        self.states = ['0', '1', '2']
        self.transitions = {'0': ['0', '1'], '1': ['0', '1', '2'], '2': ['2']}
        self.current = '0'
        self.time = 0
        self.matrix = matrix
        self.count_for_state1 = 0
    def transition(self, stateA, stateB):
        #print('Состояние А - {}, состояние B - {}, счетчик - {}'.format(stateA, stateB, self.count_for_state1))
        if self.current == stateA:
            index = self.transitions[stateA].index(stateB)
            self.current = stateB
            if stateA == '1' and self.count_for_state1 != mttbr and stateB != '2':
                print('Процесс восстановления')
                self.current = '1'
            elif self.current != '2' and self.count_for_state1 == mttbr:
                print('СТОП! ВОССТАНОВИЛИ')
                print('Счетчик достиг пика {} и был сброшен'.format(self.count_for_state1))
                self.current = '0'
                self.count_for_state1 = 0
                #print('Теперь система в состоянии {} и счетчик {}'.format(self.current, self.count_for_state1))
            print('Я сделаль переход {} -> {}, счетчик {}'.format(stateA, self.current, self.count_for_state1))

    def make_transition(self):
        self.time += 1
        if self.current == '1':
            self.count_for_state1 += 1
        gen_prob = random.random()
        print('Текущее состояние {}, вероятность {}'.format(self.current, gen_prob))
        prev = self.current
        row = self.matrix[int(self.current)]
        print(row)
        for i in range(len(row)):
            if gen_prob < row[i]:
                print('Надо перейти в состояние {}'.format(str(i)))
                if self.count_for_state1 != mttbr:
                    self.transition(self.current, str(i))
                else:
                    if i == 2:
                        self.count_for_state1 = 0
                        self.transition(self.current, self.current)                        
                    else:
                        self.transition(self.current, str(i))
                break
    def reset(self):
        self.time = 0
        self.current = '0'
        self.count_for_state1 = 0
    def one_full_situation(self):
        self.reset()
        while (self.current != '2'):
            self.make_transition()
        print('Время жизни {}'.format(self.time))
        return self.time

In [7]:
print(matrix_intervals)
fsm228 = MyFiniteStateMachine(matrix_intervals)
fsm228.one_full_situation()

[[0.8913043478260869, 1.0, 1.0], [0.14285714285714285, 0.9021739130434783, 1.0], [0, 0, 1.0]]
Текущее состояние 0, вероятность 0.47228416612028656
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероятность 0.05333549462030773
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероятность 0.6621527881262614
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероятность 0.3172106307322813
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероятность 0.39029272295448125
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероятность 0.6881383775559939
[0.8913043478260869, 1.0, 1.0]
Надо перейти в состояние 0
Я сделаль переход 0 -> 0, счетчик 0
Текущее состояние 0, вероят

9

In [114]:
def statitics(quantity, matrix_intervals):
    summ = 0
    for i in range(quantity):
        fsm11 = MyFiniteStateMachine(matrix_intervals)
        time = fsm11.one_full_situation()
        summ += time
    return summ/quantity

In [115]:
start = time.clock()
average = statitics(1000000, matrix_intervals)
finish = time.clock()
print('Среднее время жизни пакета {}, потрачено {}с'.format(average, finish-start))

Среднее время жизни пакета 30.276159, потрачено 99.24562949495245с


In [116]:
IA = average / (average + mttbr)
print(IA)

0.8122124116918806


In [5]:
class MyFiniteStateMachineTrue(object):
    def __init__(self, matrix):
        self.states = ['0', '1', '2']
        self.transitions = {'0': ['0', '1'], '1': ['0', '1', '2'], '2': ['2']}
        self.current = '0'
        self.time = 0
        self.matrix = matrix
    def transition(self, stateA, stateB):
            self.current = stateB
            #print('Я сделаль переход {} -> {}'.format(stateA, self.current))
    def make_transition(self):
        self.time += 1
        gen_prob = random.random()
        #print('Текущее состояние {}, вероятность {}'.format(self.current, gen_prob))
        prev = self.current
        row = self.matrix[int(self.current)]
        #print(row)
        for i in range(len(row)):
            if gen_prob < row[i]:
                #print('Надо перейти в состояние {}'.format(str(i)))
                self.transition(self.current, str(i))
                break
    def reset(self):
        self.time = 0
        self.current = '0'
    def one_full_situation(self):
        self.reset()
        while (self.current != '2'):
            self.make_transition()
        #print('Время жизни {}'.format(self.time))
        return self.time

In [28]:
fsm13 = MyFiniteStateMachineTrue(matrix_intervals)
fsm13.one_full_situation()

412

In [10]:
def statisticsTrue(quantity, matrix_intervals):
    summ = 0
    for i in range(quantity):
        fsm11 = MyFiniteStateMachineTrue(matrix_intervals)
        time = fsm11.one_full_situation()
        summ += time
    return summ/quantity

In [8]:
start1 = time.clock()
average1 = statisticsTrue(1000000, matrix_intervals)
finish1 = time.clock()
print('Среднее время жизни пакета {}, потрачено {}с'.format(average1, finish1-start1))

Среднее время жизни пакета 523.546606, потрачено 1124.4815172686817с


In [18]:
start2 = time.clock()
average2 = statisticsTrue(100000, matrix_intervals)
finish2 = time.clock()
print('Среднее время жизни пакета {}, потрачено {}с'.format(average2, finish2-start2))

Среднее время жизни пакета 523.48238, потрачено 115.17813000202841с


In [30]:
IA = average1 / (average1 + mttbr)
print('Коэффициент доступности информации {}'.format(IA))

Коэффициент доступности информации 0.981257494870092


In [29]:
startfun = time.clock()
averagefun = statisticsTrue(100000, matrix_intervals)
finishfun = time.clock()
print('Среднее время жизни пакета {}, потрачено {}с'.format(averagefun, finishfun-startfun))

Среднее время жизни пакета 524.13597, потрачено 135.26340916255504с
